# import dependencies

In [14]:
import itertools
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
!pip install pykitti
import pykitti
# !pip uninstall opencv-contrib-python
!pip install opencv-contrib-python==3.4.2.16
!pip install opencv-python==3.4.2.16
!pip install opencv-python-headless==4.1.1.26

import cv2
from matplotlib.patches import Circle
import csv

import time
import os
import copy
from PIL import Image as PImage
import io
import math
from scipy.spatial import distance as dist


In [15]:
# run on Google Colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Load Kitti dataset

In [0]:
 # include the path to KITTI directory
basedir = '/content/drive/My Drive/2011_09_26_drive_0001_sync'


date = '2011_09_26'
drive = '0001'

dataset = pykitti.raw(basedir, date, drive)

## Camera displacement utils

In [0]:
def latToScale(lat):
# compute mercator scale from latitude
    return math.cos(lat * math.pi / 180)

def latlonToMercator(lat,lon,scale):
# converts lat/lon coordinates to mercator coordinates using mercator scale
    er = 6378137 # earth radius
    mx = scale * lon * math.pi * er / 180
    my = scale * er * math.log( math.tan((90+lat) * math.pi / 360) )
    return mx, my

def convertOxtsToPose(oxts):
    scale = latToScale(oxts[0][0])
    pose = []
    Tr_0_inv = np.zeros(shape=(4,4))
    Tr = np.zeros(shape=(4,4))
    t = np.zeros(shape=(3, 1))
    for i in range(len(oxts)):
        if not oxts[i]:
            pose.append([])
            continue

        # translation vector 
        t[0,0], t[1,0] = latlonToMercator(oxts[i][0],oxts[i][1],scale)
        t[2,0] = oxts[i][2]

        # rotation matrix (OXTS RT3000 user manual, page 71/92)
        rx = oxts[i][3] # roll
        ry = oxts[i][4] # pitch
        rz = oxts[i][5] # heading 
        # base => nav  (level oxts => rotated oxts)
        Rx = np.matrix([[1, 0, 0], [0, math.cos(rx), -math.sin(rx)], [0, math.sin(rx), math.cos(rx)]]) 
        # base => nav  (level oxts => rotated oxts)
        Ry =  np.matrix([[math.cos(ry), 0, math.sin(ry)], [0, 1, 0], [-math.sin(ry), 0, math.cos(ry)]]) 
        # base => nav  (level oxts => rotated oxts)
        Rz = np.matrix([[math.cos(rz), -math.sin(rz), 0], [math.sin(rz), math.cos(rz), 0], [0, 0, 1]]) 
        R  = Rz*Ry*Rx

        # print(R)
        # print(t)
        Tr[:3, :3] = R
        Tr[:3, 3:] = t
        Tr[3,3] = 1
        # normalize translation and rotation (start at 0/0/0)
        if i == 0:
            Tr_0_inv = np.linalg.inv(Tr)

        pose.append((Tr_0_inv.dot(Tr)))
    return np.stack(pose)

## Velodyne in Camera utils

In [0]:
# https://stackoverflow.com/questions/45333780/kitti-velodyne-point-to-pixel-coordinate
def prepare_velo_points(pts3d_raw):
    '''Replaces the reflectance value by 1, and tranposes the array, so
       points can be directly multiplied by the camera projection matrix'''

    pts3d = pts3d_raw
    # Reflectance > 0
    pts3d = pts3d[pts3d[:, 3] > 0 ,:]
    pts3d[:,3] = 1
    return pts3d.transpose()

def project_velo_points_in_img(pts3d, T_cam_velo, Rrect, Prect):
    '''Project 3D points into 2D image. Expects pts3d as a 4xN
       numpy array. Returns the 2D projection of the points that
       are in front of the camera only an the corresponding 3D points.'''

    # 3D points in camera reference frame.
    pts3d_cam = Rrect.dot(T_cam_velo.dot(pts3d))

    # Before projecting, keep only points with z>0 
    # (points that are in fronto of the camera).
    idx = (pts3d_cam[2,:]>=0)
    pts2d_cam = Prect.dot(pts3d_cam[:,idx])
    return pts3d[:, idx], pts2d_cam/pts2d_cam[2,:]

In [0]:
sift = cv2.xfeatures2d.SIFT_create()

In [0]:
# IX_path = '/content/drive/My Drive/TFM/ball_big.jpg'
row_list = [["x_displacement", "y_displacement", "sensors_x", "sensors_y", "sensors_d", "sensors_e", "coordinates_src", "depth_src","coordinates_dst", "depth_dst"]]


## SIFT features utils

In [0]:
def pairwise_distance(X, Y):
    assert len(X.shape) == len(Y.shape)
    N = X.shape[0]
    M = Y.shape[0]
    # D = len(X.shape)
    res = np.zeros([M, N])
    for i in range(M):
        for j in range(N):
            res[i][j] = np.linalg.norm(X[j] - Y[i])
    return res

def match(PD):
    seq = np.arange(PD.shape[0]) # create array of a certain shape from 0 to len
    amin1 = np.argmin(PD, axis=1) # return smallest index from each list
    C = np.array([seq, amin1]).transpose()
    min1 = PD[seq, amin1] # array of the minimum distances
    mask = np.zeros_like(PD) # return an array of zeros of the same shape as DP
    mask[seq, amin1] = 1
    masked = np.ma.masked_array(PD, mask) # PD without the smallest values
    min2 = np.amin(masked, axis=1) # return smallest num in list
    return C, np.array(min2/min1)

def match_max(PD):
    seq = np.arange(PD.shape[0])
    amax1 = np.argmin(PD, axis=1)
    C = np.array([seq, amax1]).transpose()
    return C

## Features with depth utils

In [0]:
def find2perCent(num, listPro):
    listfeatures = []
    Menys = 0.98 * num
    Mes = 1.02 * num 
    for i, el in enumerate(listPro):
        if el < Mes and el > Menys:
            listfeatures.append(i)
    return listfeatures

## Main loop


In [0]:
oxts = []
for pos in dataset.oxts:
    oxt = [i for i in pos[0]]
    oxts.append(oxt)

camera_displacement = []
pose = convertOxtsToPose(oxts)
l = 3 # coordinate axis length
A = np.array([[0, 0, 0, 1], [l, 0, 0, 1], [0, 0, 0, 1], [0, l, 0, 1], [0, 0, 0, 1], [0, 0, l, 1]]).transpose()

In [24]:
index = 0
frames = 5
depth = False
for j, cam3 in enumerate(dataset.cam3):
    B = pose[j].dot(A)
    pts3d = prepare_velo_points(dataset.get_velo(j))
    if j == 0:
        IX = np.array(cam3.convert('RGB'))
        sizeX = IX.shape[1]
        sizeY = IX.shape[0]
        IX = IX[:, :, ::-1].copy() 
        projectionX = project_velo_points_in_img(pts3d, dataset.calib.T_cam3_velo, dataset.calib.R_rect_30,dataset.calib.P_rect_30)

    elif j%frames ==0:
        projectionY = project_velo_points_in_img(pts3d, dataset.calib.T_cam3_velo, dataset.calib.R_rect_30,dataset.calib.P_rect_30)
        IY = np.array(cam3.convert('RGB'))
        IY = IY[:, :, ::-1].copy() 
        kp1, des1 = sift.detectAndCompute(IX, None)
        kp2, des2 = sift.detectAndCompute(IY, None)
        bf = cv2.BFMatcher()
        matches = bf.match(des1, des2)
        matches = sorted(matches, key=lambda x: x.distance)
        num_matches = len(matches)
        matches = matches[:130]
        Xscaled = []
        Yscaled = []
        for m in matches:
            Xscaled.append(kp1[m.queryIdx].pt)
            Yscaled.append(kp2[m.trainIdx].pt)
        Xscaled = np.array(Xscaled)
        Yscaled = np.array(Yscaled)
        if depth:
            featureWithDepthX = np.zeros(shape=(Xscaled.shape[0], Xscaled.shape[1] +2), dtype=np.float64)
            featureWithDepthY = np.zeros(shape=(Yscaled.shape[0], Yscaled.shape[1] +2), dtype=np.float64)

            projectionX_roundedX = [round(i, 1) for i in projectionX[1][0]]
            projectionX_roundedY = [round(i, 1) for i in projectionX[1][1]]
            projectionY_roundedX = [round(i, 1) for i in projectionY[1][0]]
            projectionY_roundedY = [round(i, 1) for i in projectionY[1][1]]

            for i, feature in enumerate(Xscaled):
                listindX =find2perCent(round(feature[0], 1), projectionX_roundedX)
                listindY =find2perCent(round(feature[1], 1), projectionX_roundedY)
                if len(listindY) > 0 and len(listindX) > 0:
                    cand = set(listindX).intersection(listindY) # detect coordindates 
                    if len(cand)>0:
                        featureWithDepthX[i] = np.append(feature, [True, projectionX[0][0][cand.pop()]])

            for i, feature in enumerate(Yscaled):
                listindX =find2perCent(round(feature[0], 1), projectionY_roundedX)
                listindY =find2perCent(round(feature[1], 1), projectionY_roundedY)
                if len(listindY) > 0 and len(listindX) > 0:
                    cand = set(listindX).intersection(listindY) # detect coordindates 
                    if len(cand)>0:
                        featureWithDepthY[i] = np.append(feature, [True, projectionY[0][0][cand.pop()]])
        else:
            featureWithDepthX = Xscaled
            featureWithDepthY = Yscaled

        difference_listX = []
        difference_listY = []
        difference_listZ = []
        euclideanDistance_list = []

        coordinatesSRC = []
        coordinatesDST = []
        depthListSRC = []
        depthListDST = []

        for i, pnt in enumerate(featureWithDepthX):
            if depth:
                if pnt[2] == True and featureWithDepthY[i][2] == True:
                    src_x = (pnt[1])
                    src_y = (pnt[0])
                    dst_x = (featureWithDepthY[i][1])
                    dst_y = (featureWithDepthY[i][0])

                    difference_listX.append(src_x - dst_x)
                    difference_listY.append(src_y - dst_y)
                    difference_listZ.append(featureWithDepthX[i][3] - featureWithDepthY[i][3])
                    euclideanDistance_list.append(dist.euclidean((src_x, src_y), (dst_x, dst_y)))

                    coordinatesSRC.append(src_x/sizeX)
                    coordinatesSRC.append(src_y/sizeY)
                    depthListSRC.append(featureWithDepthX[i][3])

                    coordinatesDST.append(dst_x/sizeX)
                    coordinatesDST.append(dst_y/sizeY)
                    depthListDST.append(featureWithDepthY[i][3])

            else:
                src_x = (pnt[1])
                src_y = (pnt[0])
                dst_x = (featureWithDepthY[i][1])
                dst_y = (featureWithDepthY[i][0])

                difference_listX.append(src_x - dst_x)
                difference_listY.append(src_y - dst_y)
                euclideanDistance_list.append(dist.euclidean((src_x, src_y), (dst_x, dst_y)))

                depthListSRC.append(1)
                coordinatesSRC.append(src_x/sizeX)
                coordinatesSRC.append(src_y/sizeY)

                depthListDST.append(1)
                coordinatesDST.append(dst_x/sizeX)
                coordinatesDST.append(dst_y/sizeY)
                

        x_displacement = B[0,0:2][0] - pose[j-frames].dot(A)[0,0:2][0]
        y_displacement = B[1,2:4][0] - pose[j-frames].dot(A)[1,2:4][0]
        index +=1
        
        row_list.append([x_displacement, y_displacement, difference_listX, difference_listY,difference_listZ, euclideanDistance_list, coordinatesSRC, depthListSRC, coordinatesDST, depthListDST])

        IX = IY
        projectionX = projectionY

## Create the CSV file

In [0]:
with open('kitti0001_SIFTDepthless-sensors.csv', 'w', newline='') as myfile:
    writer = csv.writer(myfile, quoting=csv.QUOTE_NONNUMERIC, delimiter=',')
    writer.writerows(row_list)
